# Functions

In [1]:
import torch
import numpy as np

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cpu



In [2]:
def compute_errors(gt, pred):
    thresh = np.maximum((gt / pred), (pred / gt))
    d1 = (thresh < 1.25).mean()
    d2 = (thresh < 1.25 ** 2).mean()
    d3 = (thresh < 1.25 ** 3).mean()

    rms = (gt - pred) ** 2
    rms = np.sqrt(rms.mean())

    log_rms = (np.log(gt) - np.log(pred)) ** 2
    log_rms = np.sqrt(log_rms.mean())

    abs_rel = np.mean(np.abs(gt - pred) / gt)
    sq_rel = np.mean(((gt - pred) ** 2) / gt)

    err = np.log(pred) - np.log(gt)
    silog = np.sqrt(np.mean(err ** 2) - np.mean(err) ** 2) * 100

    err = np.abs(np.log10(pred) - np.log10(gt))
    log10 = np.mean(err)

    return [silog, abs_rel, log10, rms, sq_rel, log_rms, d1, d2, d3]

# Evo dataset

## Focal dist

In [3]:
# Intrinsics matrices: (x_focal, y_focal, x_pp, y_pp)
K_maxtrces = {
    'ckad_01_ckad_2020-10-29-16-53-38_0': (
        1052.69873046875,
        1052.69873046875,
        969.119079589844,
        555.560913085938,
    ),
    'hospital_01_2020-07-14-14-03-23_0': (
        1412.705078125,
        1412.705078125,
        967.168579101562,
        543.377807617188,
    ),
    'ipcp_03_2020-06-23_ipcp_ipcp_n1-03_2020-06-23-16-54-40Z_converted.evo1a_old_default': (
        1411.85119628906,
        1411.85119628906,
        991.805480957031,
        505.537628173828,
    ),
    'ipcp_03_2020-08-19_ipcp_ipcp_n1-03_2020-08-18-10-14-48Z_converted.evo1a_old_default': (
        1418.53186035156,
        1418.53186035156,
        963.149108886719,
        525.431640625,
    ),
    'kalibr_002_2021-03-10_gnss_tests_kalibr-2_n1-002_2021-03-10-10-59-42Z_.evo1a_record_default': (
        521.493103027344,
        521.493103027344,
        630.518432617188,
        362.928863525391,
    ),
    'kalibr_002_kalibr-2_n1-002_2021-03-02-12-35-04Z_.evo1a_record_default': (
        521.493103027344,
        521.493103027344,
        630.518432617188,
        362.928863525391,
    ),
    'kalibr_002_kalibr-2_n1-002_2021-03-02-12-48-36Z_.evo1a_record_default': (
        521.493103027344,
        521.493103027344,
        630.518432617188,
        362.928863525391,
    ),
    'kalibr_03_2020-05-22_kalibr_2020-05-22-18-40-28_full_route_high_speed': (
        1410.72094726563,
        1410.72094726563,
        991.791381835938,
        505.526275634766,
    ),
    'kalibr_03_2020-06-01_kalibr_2020-06-01-16-38-16_0': (
        1411.34484863281,
        1411.34484863281,
        991.828979492188,
        505.558074951172,
    ),
    'kalibr_04_2020-12-03_2020-12-03-13-26-10_0_ZED': (
        523.876708984375,
        523.876708984375,
        647.893188476563,
        369.270965576172,
    ),
    'ckad_01_ckad_2020-10-29-17-01-56_0': (
        1052.69873046875,
        1052.69873046875,
        969.119079589844,
        555.560913085938,
    ),
    'hospital_01_2020-07-14-16-59-35_0': (
        1412.67077636719,
        1412.67077636719,
        967.1748046875,
        543.375671386719,
    ),
    'ipcp_03_2020-06-23_ipcp_2020-06-23-16-48-18_0': (
        1418.07263183594,
        1418.07263183594,
        991.793395996094,
        505.523162841797,
    ),
    'ipcp_03_2020-08-19_ipcp_2020-08-19-20-23-09_0': (
        1418.53186035156,
        1418.53186035156,
        963.149108886719,
        525.431640625,
    ),
    'kalibr_04_2021-01-18_snow_2021-01-18-15-29-39_0': (
        523.876708984375,
        523.876708984375,
        647.893188476563,
        369.270965576172,
    ),
    'kalibr_04_2021-01-18_snow_2021-01-18-15-37-39_0': (
        523.876708984375,
        523.876708984375,
        647.893188476563,
        369.270965576172,
    ),
    'kapotnya_02_2020-02-26_kapotnya_2020-02-26-15-27-06_0': (
        1409.71423339844,
        1409.71423339844,
        991.831970214844,
        505.561462402344,
    ),
    'kapotnya_02_2020-02-26_kapotnya_2020-02-26-15-09-42_0': (
        1418.07263183594,
        1418.07263183594,
        991.793395996094,
        505.523162841797,
    ),
    'kapotnya_02_2020-02-26_kapotnya_2020-02-26-15-34-18_0': (
        1409.71423339844,
        1409.71423339844,
        991.831970214844,
        505.561462402344,
    ),
    'kapotnya_02_2020-02-26_kapotnya_2020-02-26-15-42-08_0': (
        1409.71423339844,
        1409.71423339844,
        991.831970214844,
        505.561462402344,
    ),
}

## Split data

In [4]:
from pathlib import Path
import pandas as pd


evo_dir = Path("/home/dvkorshunov/mono_depth/monocular_depth_estimation/datasets/bags_historical")
data = pd.read_csv(evo_dir / "markup.csv")

In [5]:
scenes = data.image.str.rsplit("_", n=2, expand=True)[0].str.split("/", n=1, expand=True)[1].unique()

bts_data = []
for row in data.itertuples():
    scene = row.image.rsplit("_", maxsplit=2)[0].split("/")[1]
    bts_data.append({
        "image": row.image,
        "depth": row.depth_fname,
        "focal": K_maxtrces[scene][0],
        "split": row.split,
    })
    
splited = pd.DataFrame(bts_data)

In [6]:
train_fname = evo_dir / "tmp_train.txt"
columns = ["image", "depth", "focal"]
splited[splited.split == "train"][columns].to_csv(
    train_fname,
    header=False,
    index=False,
    sep=" ",
#     quoting=csv.QUOTE_NONE,
    escapechar=" ",
)

# BTS

In [7]:
import sys
bts_dir = "/home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts"
bts_torch_dir = bts_dir + "/pytorch"
bts_pretrains_dir = bts_dir + "/models"
checkpoint_path = bts_pretrains_dir + "/bts_eigen_v2_pytorch_densenet121/model"
dataset_dir = "/home/dvkorshunov/mono_depth/monocular_depth_estimation/datasets/datasets/kitti/"
sys.path.insert(1, bts_torch_dir)

In [8]:
from collections import namedtuple

arguments = (
    "encoder "
    "data_path "
    "dataset "
    "filenames_file "
    "model_name "
    "checkpoint_path "
    "input_height "
    "input_width "
    "max_depth "
    "bts_size "
    "do_kb_crop"
)
Params = namedtuple('Params', arguments)
params = Params(
    encoder="densenet121_bts",
#     data_path=dataset_dir,
    data_path=evo_dir,
    dataset="kitti",
#     filenames_file= bts_dir + "/train_test_inputs/eigen_test_files_with_gt_copy.txt",
    filenames_file=train_fname.as_posix(),
    model_name="bts_eigen_v2_pytorch_densenet121",
    checkpoint_path=checkpoint_path,
    input_height=352,
    input_width=1216,
    max_depth=80,
    bts_size=512,
    do_kb_crop=True,
)

# Load pretrains

In [9]:
! mkdir -p {bts_pretrains_dir}
! wget -nc https://cogaplex-bts.s3.ap-northeast-2.amazonaws.com/bts_eigen_v2_pytorch_densenet121.zip -O {bts_pretrains_dir + "/bts_eigen_v2_pytorch_densenet121.zip"}
! unzip -o {bts_pretrains_dir + "/bts_eigen_v2_pytorch_densenet121.zip"} -d {bts_pretrains_dir}    

File ‘/home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121.zip’ already there; not retrieving.
Archive:  /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121.zip
  inflating: /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121/arguments_train_eigen.txt  
  inflating: /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121/bts_eigen_v2_pytorch_densenet121.py  
  inflating: /home/dvkorshunov/mono_depth/monocular_depth_estimation/networks/bts/models/bts_eigen_v2_pytorch_densenet121/model  


# Profile

In [10]:
from tqdm.notebook import tqdm
from torch.autograd import Variable

from bts_dataloader import BtsDataLoader
from bts import BtsModel

def get_num_lines(file_path):
    f = open(file_path, 'r')
    lines = f.readlines()
    f.close()
    return len(lines)

dataloader = BtsDataLoader(params, mode="test")
    
model = BtsModel(params=params)
model = torch.nn.DataParallel(model)

checkpoint = torch.load(params.checkpoint_path, map_location=torch.device(device.type))
model.load_state_dict(checkpoint['model'])
model.eval()
model.to(device.type)

DataParallel(
  (module): BtsModel(
    (encoder): encoder(
      (base_model): Sequential(
        (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu0): ReLU(inplace=True)
        (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (denseblock1): _DenseBlock(
          (denselayer1): _DenseLayer(
            (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace=True)
            (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace=True)
            (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          )
          (denselayer2): _DenseLayer

In [11]:

for num, sample in enumerate(tqdm(dataloader.data)):
    image = Variable(sample['image'].to(device.type))
    focal = Variable(sample['focal'].to(device.type))

    # Predict
    lpg8x8, lpg4x4, lpg2x2, reduc1x1, depth_est = model(image, focal)
    pred_depths.append(depth_est.cpu().numpy().squeeze())
    pred_8x8s.append(lpg8x8[0].cpu().numpy().squeeze())
    pred_4x4s.append(lpg4x4[0].cpu().numpy().squeeze())
    pred_2x2s.append(lpg2x2[0].cpu().numpy().squeeze())
    pred_1x1s.append(reduc1x1[0].cpu().numpy().squeeze())

  0%|          | 0/2202 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import time

num_params = sum([np.prod(p.size()) for p in model.parameters()])
print("Total number of parameters: {}".format(num_params))

num_test_samples = get_num_lines(params.filenames_file)

with open(params.filenames_file) as f:
    lines = f.readlines()

print('now testing {} files with {}'.format(num_test_samples, params.checkpoint_path))

pred_depths = []
pred_8x8s = []
pred_4x4s = []
pred_2x2s = []
pred_1x1s = []

start_time = time.time()
profile_data = []
with torch.no_grad():
    for num, sample in enumerate(tqdm(dataloader.data)):
        image = Variable(sample['image'].to(device.type))
        focal = Variable(sample['focal'].to(device.type))
        
        # warm up
        if num == 30:
            print(profile_net(model, image, focal))
            break
        # Predict
        lpg8x8, lpg4x4, lpg2x2, reduc1x1, depth_est = model(image, focal)
        pred_depths.append(depth_est.cpu().numpy().squeeze())
        pred_8x8s.append(lpg8x8[0].cpu().numpy().squeeze())
        pred_4x4s.append(lpg4x4[0].cpu().numpy().squeeze())
        pred_2x2s.append(lpg2x2[0].cpu().numpy().squeeze())
        pred_1x1s.append(reduc1x1[0].cpu().numpy().squeeze())

elapsed_time = time.time() - start_time
print('Elapesed time: %s' % str(elapsed_time))
print('Done.')

In [ ]:
# import pandas as pd

# df = pd.DataFrame(profile_data)
# df["elapsed_time, ms"]